In [1]:
%pip install selenium

                                              0.0/9.6 MB ? eta -:--:--
                                              0.0/9.6 MB ? eta -:--:--
                                              0.0/9.6 MB 653.6 kB/s eta 0:00:15
                                              0.1/9.6 MB 1.1 MB/s eta 0:00:09
                                              0.1/9.6 MB 722.1 kB/s eta 0:00:14
                                              0.2/9.6 MB 908.0 kB/s eta 0:00:11
     -                                        0.3/9.6 MB 1.2 MB/s eta 0:00:08
     --                                       0.5/9.6 MB 1.7 MB/s eta 0:00:06
     --                                       0.6/9.6 MB 1.7 MB/s eta 0:00:06
     ---                                      0.9/9.6 MB 2.5 MB/s eta 0:00:04
     -----                                    1.2/9.6 MB 2.9 MB/s eta 0:00:03
     -----                                    1.4/9.6 MB 3.0 MB/s eta 0:00:03
     -------                                  1.7/9.6 MB 3.3 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.1.1 requires sentencepiece, which is not installed.
botocore 1.27.59 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.5.0 which is incompatible.
requests 2.29.0 requires urllib3<1.27,>=1.21.1, but you have urllib3 2.5.0 which is incompatible.


In [45]:
import os
import time
import traceback
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==== CONFIGURAÇÃO DE PASTA DE DOWNLOAD ====
download_dir = os.path.abspath("csvs")
os.makedirs(download_dir, exist_ok=True)

# ==== CONFIGURAÇÃO DO CHROME ====
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
options.add_argument("--headless")  # Rode sem abrir o navegador (opcional)

# ==== ABRE NAVEGADOR ====
driver = webdriver.Chrome(options=options)

# ==== URL base ====
URL = "https://apps.tre-ce.jus.br/tre/consulta-votacao/2045202024/13897/13"
driver.get(URL)

erros= []

try:
    # Espera os cards carregarem
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
    )

    # Pega todos os links de cards
    cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
    total = len(cards)

    print(f"🔍 Encontrados {total} candidatos.")

    for i in range(499, total):
        try:
            print(f"\n➡️ Processando candidato {i+1}/{total}...")

            # Recarrega a página toda vez para evitar stale elements
            driver.get(URL)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
            )
            cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
            cards[i].click()

            # Espera botão e clica em "Votação por seção"
            botao_secao = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Votação por Seção')]"))
            )
            time.sleep(1)
            botao_secao.click()

            # Espera botão de download e clica
            botao_csv = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ui.green.tiny.left.attached.button.ml-2"))
            )
            botao_csv.click()

            # Aguarda o download terminar
            time.sleep(5)

            # Nome do candidato
            nome_completo = driver.find_element(By.CLASS_NAME, "custom-header-card").text
            nome_candidato = nome_completo.split(" - ")[1].strip().replace("\n", " ")
            nome_candidato = re.sub(r'[\\/*?:"<>|\n]', '', nome_candidato)

            # Renomear último arquivo baixado
            arquivos = sorted(
                os.listdir(download_dir),
                key=lambda x: os.path.getmtime(os.path.join(download_dir, x)),
                reverse=True
            )
            arquivo_baixado = arquivos[0]
            novo_nome = f"{nome_candidato}.csv"
            os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))

            print(f"✅ CSV salvo como: {novo_nome}")

        except Exception as e:
            print(f"⚠️ Erro ao processar candidato {i+1}: {e}")
            erros.append(i+1)
            traceback.print_exc()
            continue

except Exception as e:
    print(f"❌ Erro geral: {e}")
    traceback.print_exc()

finally:
    driver.quit()
    print("\n🏁 Processo finalizado.")
    print(list(erros))


🔍 Encontrados 746 candidatos.

➡️ Processando candidato 500/746...
✅ CSV salvo como: NESTOR BEZERRA NESTOR BEZERRA.csv

➡️ Processando candidato 501/746...
✅ CSV salvo como: NETO DA NÓBREGA JOSÉ FERREIRA DA NÓBREGA NETO.csv

➡️ Processando candidato 502/746...
✅ CSV salvo como: NETO DOS RELÓGIOS FRANCISCO NETO DA COSTA.csv

➡️ Processando candidato 503/746...
✅ CSV salvo como: NEUDIAN COSTA JOSÉ NEUDIAN DA COSTA.csv

➡️ Processando candidato 504/746...
✅ CSV salvo como: NEY MAIA NEY ISMAYLE MAIA FACUNDO.csv

➡️ Processando candidato 505/746...
✅ CSV salvo como: NIERTON GUERRA ANTONIO NIERTON QUEIROZ GUERRA.csv

➡️ Processando candidato 506/746...
✅ CSV salvo como: NILO DANTAS ANDERSON NILO DANTAS VIANA.csv

➡️ Processando candidato 507/746...
✅ CSV salvo como: NORA BARRETO FRANCISCA ALDENORA DO NASCIMENTO SILVA.csv

➡️ Processando candidato 508/746...
✅ CSV salvo como: NOVIM JUAREZ PRECEBES MOURA.csv

➡️ Processando candidato 509/746...
✅ CSV salvo como: NUBIA LAFAIETE NÚBIA LAFAIETE G

In [44]:
import os
import time
import traceback
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==== CONFIGURAÇÃO DE PASTA DE DOWNLOAD ====
download_dir = os.path.abspath("csvs")
os.makedirs(download_dir, exist_ok=True)

# ==== CONFIGURAÇÃO DO CHROME ====
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
options.add_argument("--headless")  # Rode sem abrir o navegador (opcional)

# ==== ABRE NAVEGADOR ====
driver = webdriver.Chrome(options=options)

# ==== URL base ====
URL = "https://apps.tre-ce.jus.br/tre/consulta-votacao/2045202024/13897/13"
driver.get(URL)

try:
    # Espera os cards carregarem
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
    )

    # Pega todos os links de cards
    cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
    total = len(cards)

    print(f"🔍 Encontrados {total} candidatos.")
    valor_errado = 487
  
    try:
        print(f"\n➡️ Processando candidato {valor_errado+1}...")

        # Recarrega a página toda vez para evitar stale elements
        driver.get(URL)
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
        )
        cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
        cards[valor_errado].click()

        # Espera botão e clica em "Votação por seção"
        botao_secao = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Votação por Seção')]"))
        )
        time.sleep(1)
        botao_secao.click()

        # Espera botão de download e clica
        botao_csv = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ui.green.tiny.left.attached.button.ml-2"))
        )
        botao_csv.click()

        # Aguarda o download terminar
        time.sleep(5)

        # Nome do candidato
        nome_completo = driver.find_element(By.CLASS_NAME, "custom-header-card").text
        nome_candidato = nome_completo.split(" - ")[1].strip().replace("\n", " ")
        nome_candidato = re.sub(r'[\\/*?:"<>|\n]', '', nome_candidato)

        # Renomear último arquivo baixado
        arquivos = sorted(os.listdir(download_dir), key=lambda x: os.path.getmtime(os.path.join(download_dir, x)),reverse=True)
        arquivo_baixado = arquivos[0]
        novo_nome = f"{nome_candidato}.csv"
        os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))

        print(f"✅ CSV salvo como: {novo_nome}")

    except Exception as e:
        print(f"⚠️ Erro ao processar candidato {valor_errado}: {e}")
        traceback.print_exc()

except Exception as e:
    print(f"❌ Erro geral: {e}")
    traceback.print_exc()

finally:
    driver.quit()
    print("\n🏁 Processo finalizado.")

🔍 Encontrados 746 candidatos.

➡️ Processando candidato 488...
✅ CSV salvo como: NADIJANE NOSSA VOZ NADIJANE MARIA OLIVEIRA DE MENEZES.csv

🏁 Processo finalizado.
